# Compare Peer Companies using XBRL Data

The purpose of this notebook is to scrape Edgar filings and use the underlying XBRL data to compare between companies. This results is a horizontal comparison between companies with similar concepts and facts. For example, in this case we are looking at two Companies that occupy the digital infrastructure space.

In [1]:
from Helpers.XBRL import XBRLReport
import plotly.express as px
import pandas as pd

In [2]:
filing_url = "https://www.sec.gov/ix?doc=/Archives/edgar/data/1679688/000167968821000085/dbrg-20210630.htm"
peer_filing_url = "https://www.sec.gov/ix?doc=/Archives/edgar/data/1101239/000162828021015030/eqix-20210630.htm"

# Instantiate
## Load both the Company of interest filing and one peer filing 

In [3]:
report = XBRLReport.XBRLReport(filing_url)
report.append_report(peer_filing_url)

Initial report successfully loaded!
Additional report successfully appended!


# Prepare data

We first drop duplicates in order to successfully unstack the dataframe later on. This will remove certain concepts from our analysis, but it is necessary in order to compare between companies. The amount of concepts that will be removed shouldn't be large, as these are most likely specific concepts that should have had additional dimensions added to it. 

We then set the index so that we know each row is a unique item (i.e., unique subset of ticker, name, dimension and date). Once we have each row as a unique fact and date, then we can move the Ticker back to the column. 

In [4]:
df = report.facts_df.copy()
df = df.drop_duplicates(subset=["name", "dimensions", "date", "Ticker"], keep=False)
df = df.set_index(["Ticker", "name", "dimensions", "date"])[["value"]]
unstacked = df.sort_values(["name", "dimensions", "date"]).unstack(level="Ticker")
unstacked.columns = unstacked.columns.droplevel(0)

In [5]:
tickers = unstacked.columns.tolist()
for ticker in tickers:
    unstacked[ticker] = pd.to_numeric(unstacked[ticker], errors="coerce")

# Remove rows (i.e., concepts) where both companies don't have a value 
# Make sure all companies have similar concepts before dropping all willy nilly
unstacked = unstacked.dropna(how='any').reset_index()

# Ad Hoc Analysis

Compare concepts between companies. Need to search for concepts. 

In [6]:
unstacked\
    .sample(10)\
    .style\
    .format({col: '{:,.0f}'\
    .format for col in unstacked.select_dtypes("number").columns })

Ticker,name,dimensions,date,DBRG,EQIX
104,StockholdersEquity,,06/30/2021,"2,108,952,000","10,646,472,000"
36,EquityMethodInvestments,,12/31/2020,"574,511,000","163,071,000"
117,StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest,us-gaap:StatementEquityComponentsAxis [us-gaap:AdditionalPaidInCapitalMember],03/31/2020,"7,532,213,000","12,893,455,000"
43,FinanceLeaseLiability,,06/30/2021,"145,178,000","2,148,326,000"
38,EquitySecuritiesFvNi,us-gaap:FairValueByFairValueHierarchyLevelAxis [us-gaap:FairValueInputsLevel1Member] us-gaap:FairValueByMeasurementFrequencyAxis [us-gaap:FairValueMeasurementsRecurringMember],12/31/2020,"218,485,000","159,000"
6,CashAndCashEquivalentsAtCarryingValue,,06/30/2020,"1,099,467,000","4,785,050,000"
120,StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest,us-gaap:StatementEquityComponentsAxis [us-gaap:AdditionalPaidInCapitalMember],06/30/2021,"7,622,382,000","15,360,726,000"
17,ComprehensiveIncomeNetOfTaxIncludingPortionAttributableToNoncontrollingInterest,,04/01/2021 to 06/30/2021,"-114,586,000","136,232,000"
77,OtherComprehensiveIncomeLossCashFlowHedgeGainLossAfterReclassificationAndTax,,01/01/2021 to 06/30/2021,"1,285,000","23,778,000"
97,RetainedEarningsAccumulatedDeficit,,06/30/2021,"-6,601,522,000","1,985,003,000"


In [7]:
unstacked.query("name.str.contains('income|loss', case=False)")\
    .head()\
    .style\
    .format({col: '{:,.0f}'\
    .format for col in unstacked.select_dtypes("number").columns })

Ticker,name,dimensions,date,DBRG,EQIX
0,AccumulatedOtherComprehensiveIncomeLossNetOfTax,,06/30/2021,"83,675,000","-941,114,000"
1,AccumulatedOtherComprehensiveIncomeLossNetOfTax,,12/31/2020,"122,123,000","-913,368,000"
10,ComprehensiveIncomeNetOfTax,,01/01/2020 to 06/30/2020,"-2,369,714,000","46,418,000"
11,ComprehensiveIncomeNetOfTax,,01/01/2021 to 06/30/2021,"-407,477,000","196,955,000"
12,ComprehensiveIncomeNetOfTax,,04/01/2020 to 06/30/2020,"-1,996,141,000","199,682,000"


# Visualizations

In [8]:
px.bar(unstacked.query("name.str.contains('Asset')"), x="name", y="DBRG", color="date", barmode="group")